In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm
import pickle
import matplotlib.pyplot as plt

In [ ]:
data_file = './results.txt'
df = pd.read_csv(data_file, sep='\t', index_col=0)
df.head()

In [ ]:
df.index = df.columns
rows = []
for i in df.index:
    for c in df.columns:
        if i == c: continue
        score = df.loc[i, c]
        try:
            score = [int(row) for row in score.split('–')]
            rows.append([i, c, score[0], score[1]])
        except:
            continue
df = pd.DataFrame(rows, columns = ['home', 'away', 'home_score', 'away_score'])
df.head()

In [ ]:
teams = df.home.unique()
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index
teams.head()

In [ ]:
df = pd.merge(df, teams, left_on='home', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_home'}).drop('team', 1)
df = pd.merge(df, teams, left_on='away', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_away'}).drop('team', 1)
df.head()

In [ ]:
observed_home_goals = df.home_score.values
observed_away_goals = df.away_score.values
home_team = df.i_home.values
away_team = df.i_away.values
num_teams = len(df.i_home.unique())
num_games = len(home_team)

In [ ]:
g = df.groupby('i_away')
att_starting_points = np.log(g.away_score.mean())
g = df.groupby('i_home')
def_starting_points = -np.log(g.away_score.mean())

In [ ]:
with pm.Model() as model:
    home = pm.Normal('home', 0., .0001, testval=0)
    intercept = pm.Normal('intercept', 0., .0001, testval=0)
    tau_att = pm.Gamma('tau_att', .1, .1, testval=10)
    tau_def = pm.Gamma('tau_def', .1, .1, testval=10)
    atts_star = pm.Normal('atts_star', 0., tau_att, shape=num_teams, testval=att_starting_points)
    defs_star = pm.Normal('defs_star', 0., tau_def, shape=num_teams, testval=def_starting_points)
    atts = pm.Deterministic('atts', atts_star - atts_star.mean())
    defs = pm.Deterministic('defs', defs_star - defs_star.mean())
    home_theta = pm.Deterministic('home_theta', pm.math.exp(intercept + home + atts[home_team] + defs[away_team]))
    away_theta = pm.Deterministic('away_theta', pm.math.exp(intercept + atts[away_team] + defs[home_team]))
    home_goals = pm.Poisson('home_goals', home_theta, observed=observed_home_goals)
    away_goals = pm.Poisson('away_goals', away_theta, observed=observed_away_goals)

In [ ]:
with model:
    trace = pm.sample(5000, tune=1000, init='advi_map', nuts_kwargs=dict(target_accept=.90))

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump({'model':model, 'trace':trace}, file)

In [ ]:
df_avg = pd.DataFrame({'avg_att': trace['atts'].mean(0),
                       'avg_def': trace['defs'].mean(0)}, 
                      index=teams.team.values)
df_avg = pd.merge(df_avg, teams, left_index=True, right_on='team', how='left')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_avg.avg_att.values, df_avg.avg_def.values)
for label, x, y in zip(df_avg.team.values, df_avg.avg_att.values, df_avg.avg_def.values):
    ax.annotate(label, xy=(x,y), xytext = (-5,5), textcoords = 'offset points')
ax.set_xlabel('Avg attack effect')
ax.set_ylabel('Avg defense effect')
_ = ax.axis('equal')
plt.savefig('effect.png')